<a href="https://colab.research.google.com/github/SampMark/Recommendation-Systems/blob/main/Content_Based_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sistema de Recomendação Baseado em Conteúdo**

Sistemas de recomendação baseados em conteúdo focam em compreender as preferências individuais do usuário, construindo um perfil a partir de suas interações e avaliações anteriores. Por exemplo, se um usuário demonstrar interesse por filmes de ação com super-heróis, o sistema recomendará outros títulos com características semelhantes.

Uma das vantagens desse tipo de sistema é a personalização, já que as recomendações são geradas com base no gosto específico do usuário e nas características dos itens disponíveis. O sistema aprende as preferências do usuário e entrega recomendações altamente personalizadas.

No entanto, esse modelo apresenta algumas limitações significativas, entre elas:

* **Recomendações limitadas**: como o sistema não considera as opiniões de outros usuários, poderá sugerir itens de qualidade inferior ou restringir o acesso a novas opções, limitando a descoberta.
* **Identificação de preferências**: determinar com precisão quais aspectos dos itens agradam ao usuário nem sempre é uma tarefa simples.
* **Dificuldades na manipulação de dados**: especialmente em domínios com dados complexos ou não estruturados, tornando-se desafiador identificar e processar características relevantes em milhões de itens.

Para mitigar essas limitações, técnicas complementares, como a filtragem colaborativa, podem ser integradas. Normalmente abordagens híbridas são aplicadas na prática, combinando-se os benefícios de diferentes modelos.

Em linhas gerais, os sistemas de recomendação podem ser implementados de diferentes maneiras:

* **Abordagens baseadas em memória:** Utilizam todo o conjunto de dados de interações entre usuários e itens para gerar recomendações em tempo real, aplicando técnicas estatísticas como correlação de Pearson e similaridade de cosseno, por exemplo.
* **Abordagens baseadas em modelo:**  Criam um modelo preditivo do comportamento do usuário com base em dados históricos, utilizando técnicas de aprendizado de máquina como regressão, classificação e agrupamento.

A escolha da melhor abordagem para um sistema de recomendação depende das necessidades específicas de cada aplicação, da disponibilidade de dados e dos recursos computacionais. Neste estudo, será abordada a implementação de um sistema de recomendação simples, baseado em conteúdo.

## **Importando as bibliotecas**

---



In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

## **Explorando o Dataset do MovieLens**

O conjunto de dados explorado é do [**MovieLens Datasets**](https://dl.acm.org/doi/10.1145/2827872), o qual traz avaliações de filmes coletadas pelo Grupo de Pesquisa GroupLens na Universidade de Minnesota.

O conjunto de dados contém mais de 22,8 milhões de avaliações de 34,2 mil filmes, é um dos conjuntos de dados mais populares usados na pesquisa de sistemas de recomendação.

In [2]:
movies_df = pd.read_csv('/content/ml-latest/movies.csv')

# Informações gerais sobre o dataset
print("\nVisão geral do dataset:")
print(f"- Número de linhas: {movies_df.shape[0]}")
print(f"- Número de colunas: {movies_df.shape[1]}")

# Verificar as primeiras linhas
print("\nPrimeiras 5 linhas do dataset:")
movies_df.head()


Visão geral do dataset:
- Número de linhas: 34208
- Número de colunas: 3

Primeiras 5 linhas do dataset:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# Mostrar informações sobre colunas e tipos de dados
print("\nInformações das colunas:")
print(movies_df.info())


Informações das colunas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34208 entries, 0 to 34207
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  34208 non-null  int64 
 1   title    34208 non-null  object
 2   genres   34208 non-null  object
dtypes: int64(1), object(2)
memory usage: 801.9+ KB
None


In [4]:
ratings_df = pd.read_csv('/content/ml-latest/ratings.csv')

# Informações gerais sobre o dataset
print("\nVisão geral do dataset:")
print(f"- Número de linhas: {ratings_df.shape[0]}")
print(f"- Número de colunas: {ratings_df.shape[1]}")

# Verificar as primeiras linhas
print("\nPrimeiras 5 linhas do dataset:")
ratings_df.head()


Visão geral do dataset:
- Número de linhas: 22884377
- Número de colunas: 4

Primeiras 5 linhas do dataset:


,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [5]:
# Mostrar informações sobre colunas e tipos de dados
print("\nInformações das colunas:")
print(ratings_df.info())


Informações das colunas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22884377 entries, 0 to 22884376
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 698.4 MB
None


## **Pré-processamento dos dados**

---



In [6]:
# Extraindo para uma coluna o ano armazenado entre parênteses
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)', expand=False)

# Removendo o ano do título e limpando espaços
movies_df['title'] = movies_df['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()

# Visualizando o DataFrame atualizado
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [7]:
# Separar os gêneros em listas de strings, lidando com possíveis valores nulos
movies_df['genres'] = movies_df['genres'].fillna('').str.split('|')
# movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [8]:
# Criar uma cópia do DataFrame original
moviesWithGenres_df = movies_df.copy()

# Expandir as listas de gêneros para criar uma linha por gênero
expanded_genres = moviesWithGenres_df.explode('genres')

# Criar colunas binárias para cada gênero
genre_dummies = pd.get_dummies(expanded_genres['genres'])

# Combinar os gêneros codificados com o DataFrame original
moviesWithGenres_df = moviesWithGenres_df.join(genre_dummies.groupby(level=0).sum())

# Visualizar os primeiros registros
moviesWithGenres_df.head()

,movieId,title,genres,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,[Comedy],1995,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Remover a coluna 'timestamp'
ratings_df = ratings_df.drop(columns=['timestamp'])

# Visualizar os primeiros registros do DataFrame
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


## **Implementação do Sistema de Recomendação**

Nesta etapa, será explorada a implementação de um sistema de recomendação baseado em conteúdo, também conhecidos como sistemas item-item. O intuito é descobrir os gêneros de filme favoritos de um determinado usuário (gosto pessoal), com base em suas classificações.

Essa técnica busca identificar os aspectos preferidos de um usuário, como o gênero de filme, e então recomendar outros itens que compartilhem desses aspectos similares.  

A seguir, criei meu perfil de usuário fictício, com apenas filmes que ficaram na minha memória e as avaliações hipotéticas que fiz deles.

Caso pretenda replicar o estudo, fique à vontade para adicionar mais filmes, são mais de 34 mil nesta base, bastando aumentar a quantidade de elementos na lista.

Observações:
* Os nomes dos filmes devem escritos em inglês; e
* caso o filme comece com "The", como em "The Matrix", escreva "Matrix, The".

In [10]:
# Dados de entrada fornecidos pelo usuário
userInput = [
    {"title": "Titanic", "rating": 4.8},
    {"title": "Schindler's List", "rating": 4.5},
    {"title": "Beautiful Mind, A", "rating": 4.6},
    {"title": "Pursuit of Happyness, The", "rating": 4.0},
    {"title": "Last Samurai, The", "rating": 4.0},
    {"title": "Finding Nemo", "rating": 3.8},
]

# Validação para garantir que todos os itens têm as chaves esperadas
required_keys = {"title", "rating"}
for item in userInput:
    if not required_keys.issubset(item):
        raise ValueError(f"Chaves ausentes na entrada: {item}")

# Criação do DataFrame a partir dos dados do usuário
inputMovies = pd.DataFrame(userInput)

# Exibir o DataFrame
inputMovies

,title,rating
0,Titanic,4.8
1,Schindler's List,4.5
2,"Beautiful Mind, A",4.6
3,"Pursuit of Happyness, The",4.0
4,"Last Samurai, The",4.0
5,Finding Nemo,3.8


### **Adicionando o ID `(movieId`) do Filme ao Usuário de Entrada**

Com o perfil de usuário completo, iremos extrair os IDs dos filmes presentes no dataframe `movies_df` e adicioná-los ao perfil.

Para isso, filtraremos as linhas do dataframe que contêm os títulos dos filmes informados pelo usuário e mesclaremos esse subconjunto com o dataframe do perfil. Em seguida, removeremos as colunas desnecessárias do perfil para otimizar o uso da memória.

In [11]:
# Filtra o dataframe 'movies_df' para incluir apenas os filmes que estão presentes na lista de filmes de entrada.
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

# Faz um merge dos dataframes 'inputId' e 'inputMovies' com base na coluna 'title', combinando as informações dos filmes de entrada com seus respectivos IDs.
inputMovies = pd.merge(inputId, inputMovies, on='title')

# Remove as colunas 'genres' e 'year' do dataframe 'inputMovies', pois não são necessárias para o processo de recomendação.
inputMovies = inputMovies.drop(['genres', 'year'], axis=1)

# Exibe o dataframe 'inputMovies' resultante, que contém os filmes de entrada com seus IDs e avaliações, sem as colunas removidas.
inputMovies

,movieId,title,rating
0,527,Schindler's List,4.5
1,1721,Titanic,4.8
2,3404,Titanic,4.8
3,4995,"Beautiful Mind, A",4.6
4,6377,Finding Nemo,3.8
5,7143,"Last Samurai, The",4.0
6,47099,"Pursuit of Happyness, The",4.0
7,118916,Titanic,4.8


In [12]:
# Validar a existência das colunas necessárias
required_columns_movies = {'title', 'genres', 'year'}
required_columns_input = {'title', 'rating'}

if not required_columns_movies.issubset(movies_df.columns):
    raise ValueError(f"movies_df está faltando colunas: {required_columns_movies - set(movies_df.columns)}")
if not required_columns_input.issubset(inputMovies.columns):
    raise ValueError(f"input_movies está faltando colunas: {required_columns_input - set(inputMoviess.columns)}")

# Filtrar os filmes que correspondem aos títulos fornecidos
filtered_movies = movies_df[movies_df['title'].isin(inputMovies['title'])]

# Mesclar os DataFrames baseados no título
merged_movies = pd.merge(filtered_movies, inputMovies, on='title')

# Remover colunas desnecessárias
merged_movies = merged_movies.drop(columns=['genres', 'year'])

# Exibir o DataFrame resultante
merged_movies

,movieId_x,title,movieId_y,rating
0,527,Schindler's List,527,4.5
1,1721,Titanic,1721,4.8
2,1721,Titanic,3404,4.8
3,1721,Titanic,118916,4.8
4,3404,Titanic,1721,4.8
5,3404,Titanic,3404,4.8
6,3404,Titanic,118916,4.8
7,4995,"Beautiful Mind, A",4995,4.6
8,6377,Finding Nemo,6377,3.8
9,7143,"Last Samurai, The",7143,4.0


### **Identificando as Preferências do Usuário**

Para começar, vamos analisar as preferências do usuário. Para isso, extrairemos do dataframe o subconjunto de filmes que o usuário assistiu, com os gêneros representados por valores binários (0 ou 1).

O DataFrame `filtered_user_movies` conterá apenas os filmes em `moviesWithGenres_df` cujos IDs correspondem aos IDs fornecidos pelo usuário em `inputMovies`.

In [13]:
# Validar a existência da coluna 'movieId' em ambos os DataFrames
if 'movieId' not in moviesWithGenres_df.columns:
    raise ValueError("A coluna 'movieId' está ausente em moviesWithGenres_df.")
if 'movieId' not in inputMovies.columns:
    raise ValueError("A coluna 'movieId' está ausente em inputMovies.")

# Filtrar os filmes com base nos IDs fornecidos pelo usuário
filtered_user_movies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'])]

# Exibir o DataFrame resultante
filtered_user_movies

,movieId,title,genres,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
523,527,Schindler's List,"[Drama, War]",1993,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1661,1721,Titanic,"[Drama, Romance]",1997,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3317,3404,Titanic,"[Action, Drama]",1953,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4900,4995,"Beautiful Mind, A","[Drama, Romance]",2001,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6272,6377,Finding Nemo,"[Adventure, Animation, Children, Comedy]",2003,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7032,7143,"Last Samurai, The","[Action, Adventure, Drama, War]",2003,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11194,47099,"Pursuit of Happyness, The",[Drama],2006,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25533,118916,Titanic,"[Action, Drama, Romance]",1996,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [14]:
filtered_user_movies = filtered_user_movies.reset_index(drop=True)
userGenreTable = filtered_user_movies.drop(columns=['movieId', 'title', 'genres', 'year'])
userGenreTable

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [15]:
# Reiniciar o índice do DataFrame para garantir continuidade após filtragem
filtered_user_movies = filtered_user_movies.reset_index(drop=True)

# Verificar se as colunas a serem removidas existem no DataFrame
columns_to_drop = ['movieId', 'title', 'genres', 'year']
missing_columns = [col for col in columns_to_drop if col not in filtered_user_movies.columns]
if missing_columns:
    raise ValueError(f"As colunas ausentes no DataFrame: {missing_columns}")

# Criar a tabela de gêneros do usuário, removendo as colunas desnecessárias
user_genre_table = filtered_user_movies.drop(columns=columns_to_drop)

# Exibir o DataFrame resultante
user_genre_table

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


### **Aprendendo as Preferências do Usuário**

Agora, estamos prontos para iniciar o aprendizado das preferências do usuário!

Para isso, vamos transformar cada gênero em pesos. Isso pode ser feito utilizando as avaliações do usuário e multiplicando-as pela tabela de gêneros. Em seguida, somaremos os valores resultantes por coluna. Essa operação é equivalente ao produto escalar entre uma matriz e um vetor, que podemos calcular facilmente com a função "dot" do Pandas.

In [16]:
inputMovies['rating']

,rating
0,4.5
1,4.8
2,4.8
3,4.6
4,3.8
5,4.0
6,4.0
7,4.8


In [23]:
# Calcular o perfil do usuário com base nas classificações por gênero
user_profile = user_genre_table.T.dot(inputMovies['rating'])

# Exibir o perfil do usuário calculado
user_profile

,0
(no genres listed),0.0
Action,13.6
Adventure,7.8
Animation,3.8
Children,3.8
Comedy,3.8
Crime,0.0
Documentary,0.0
Drama,31.5
Fantasy,0.0


Agora, temos os pesos para cada uma das preferências do usuário. Isso é conhecido como Perfil do Usuário. Usando isso, podemos recomendar filmes que satisfaçam as preferências do usuário.

Vamos começar extraindo a tabela de gênero do dataframe original:

In [24]:
# Define a coluna 'movieId' como índice do DataFrame 'moviesWithGenres_df'.
genre_table = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])

# Remove as colunas 'movieId', 'title', 'genres' e 'year' do DataFrame 'genre_table'.
genre_table = genre_table.drop(columns=['movieId', 'title', 'genres', 'year'])

# Imprime o número de linhas e colunas do DataFrame 'genre_table'.
print(genre_table.shape)

# Exibe as primeiras 5 linhas do DataFrame 'genre_table'.
genre_table.head()

(34208, 20)


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
# Calcula a pontuação de recomendação para cada filme.
recommendationTable_df = ((genre_table * user_profile).sum(axis=1)) / (user_profile.sum())

# Exibe as 5 primeiras linhas do DataFrame 'recommendationTable_df'.
recommendationTable_df.head()

,0
movieId,
1,0.220690
2,0.133333
3,0.206897
4,0.568966
5,0.043678


In [26]:
# Ordena o DataFrame 'recommendationTable_df' em ordem decrescente de pontuação de recomendação.
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)

# Exibe as 5 primeiras linhas do DataFrame 'recommendationTable_df'.
recommendationTable_df.head()

,0
movieId,
26236,0.912644
74404,0.868966
8403,0.868966
74406,0.868966
74886,0.868966


In [27]:
# Filtra o DataFrame 'movies_df' para exibir os 30 filmes com as maiores pontuações de recomendação.
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(30).keys())]

,movieId,title,genres,year
149,151,Rob Roy,"[Action, Drama, Romance, War]",1995
1198,1224,Henry V,"[Action, Drama, Romance, War]",1989
1842,1925,Wings,"[Action, Drama, Romance, War]",1927
2643,2728,Spartacus,"[Action, Drama, Romance, War]",1960
4861,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th...",1980
7801,8403,Helen of Troy,"[Action, Adventure, Drama, Romance, War]",1956
7816,8446,Sands of Iwo Jima,"[Action, Drama, Romance, War]",1949
8710,26236,"White Sun of the Desert, The (Beloe solntse pu...","[Action, Adventure, Comedy, Drama, Romance, War]",1970
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
9743,31617,El Cid,"[Action, Adventure, Drama, Romance, War]",1961


# **Referências**

  F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=<http://dx.doi.org/10.1145/2827872>